# LMNA Lipodystrophy, familial partial, type 2 	151660 
Data taken from  [Lanktree M, et al. (2007) Novel LMNA mutations seen in patients with familial partial lipodystrophy subtype 2 (FPLD2; MIM 151660). Clin Genet ;71(2):183-6. PMID:17250669](https://pubmed.ncbi.nlm.nih.gov/17250669/).

The authors write:
The clinical and biochemical attributes of the three patients are shown in Table 1. Each had typical loss of subcutaneous adipose in the limbs and gluteal region, although these findings were less pronounced for patient C, as often is the case with men. Each had insulin resistance and type 2 diabetes mellitus although none had acanthosis nigricans. Each had high triglycerides and depressed high-density lipoprotein cholesterol. No findings of other laminopathies were present in any patient.

In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', None) # show entire column contents, important!
from IPython.display import display, HTML
from pyphetools.creation import *
from pyphetools.visualization import *
from pyphetools.validation import *
import pyphetools
print(f"Using pyphetools version {pyphetools.__version__}")

Using pyphetools version 0.9.65


In [2]:
PMID = "PMID:18256394"
title = "Phenotype and course of Hutchinson-Gilford progeria syndrome"
cite = Citation(pmid=PMID, title=title)
parser = HpoParser(hpo_json_file="../hp.json")
hpo_cr = parser.get_hpo_concept_recognizer()
hpo_version = parser.get_version()
hpo_ontology = parser.get_ontology()
metadata = MetaData(created_by="ORCID:0000-0002-0736-9199", citation=cite)
metadata.default_versions_with_hpo(version=hpo_version)
print(f"HPO version {hpo_version}")

HPO version 2024-02-27


In [3]:
df = pd.read_csv("input/lanktree.tsv", sep='\t')
df.head()

,Patient,Age,Variant,Onset,Onset term,Sex,BMI,Waist,Facial fat,lipodystrophy.arm,...,Diabetes.onset,HTN,NM_170707.4,Acanthosis nigricans,cholesterol.total,HDL,LDL,triglycerides,blood.glucose,HBA1Cg
0,A,46,p.D230N,Youth,Juvenile onset,F,29.3,86,excess,loss,...,44,No,c.688G>A,No,5.13,0.93,2.75,3.20,9.8,7.2
1,B,50,p.R399C,Puberty,Juvenile onset,F,35.0,80,excess,loss,...,48,No,c.1195C>T,No,4.50,1.12,2.32,2.35,6.5,7.0
2,C,75,p.S573L,Adulthood,Adult onset,M,35.6,100,excess,loss,...,57,Yes(71),c.1718C>T,No,6.04,1.14,4.10,1.75,22.7,9.5


In [4]:
#res = OptionColumnMapper.autoformat(df=df, concept_recognizer=hpo_cr)
#print(res)

In [5]:
column_mapper_list = list()
# Each value is above normal limit
bmi_d = {'29.3': 'Obesity',
 '35.0': 'Obesity',
 '35.6': 'Obesity'}
bmiMapper = OptionColumnMapper(column_name='BMI',concept_recognizer=hpo_cr, option_d=bmi_d)
column_mapper_list.append(bmiMapper)
bmiMapper.preview_column(df)

,mapping,count
0,Obesity (HP:0001513) (observed),3


In [6]:
# Facial fat = Increased facial adipose tissue HP:0000287
facialFatMapper = ConstantColumnMapper(column_name="Facial fat",hpo_id="HP:0000287", hpo_label="Increased facial adipose tissue")
column_mapper_list.append(facialFatMapper)
facialFatMapper.preview_column(df)

,mapping,count
0,excess -> observed,3


In [7]:
# Loss of subcutaneous adipose tissue in limbs HP:0003635
subqArmsMapper = ConstantColumnMapper(column_name="lipodystrophy.arm",
                                      hpo_id="HP:0003635", hpo_label="Loss of subcutaneous adipose tissue in limbs")
column_mapper_list.append(subqArmsMapper)
#subqArmsMapper.preview_column(df)

In [8]:
# Lipodystrophy HP:0009125
# Currently no more detail available
ldMapper = ConstantColumnMapper(column_name="lipodystrophy.arm",hpo_id="HP:0009125", hpo_label="Lipodystrophy")
column_mapper_list.append(ldMapper)
#ldMapper.preview_column(df)

In [9]:
# Acanthosis nigricans HP:0000956
anMapper = ConstantColumnMapper(column_name="Acanthosis nigricans",hpo_id="HP:0000956", hpo_label="Acanthosis nigricans", excluded=True)
column_mapper_list.append(anMapper)
#anMapper.preview_column(df)

In [10]:
# Type II diabetes mellitus  HP:0005978
dmMapper = ConstantColumnMapper(column_name="Diabetes.onset",hpo_id="HP:0005978", hpo_label="Type II diabetes mellitus")
column_mapper_list.append(dmMapper)
#dmMapper.preview_column(df)

In [11]:
# Hypertriglyceridemia HP:0002155
htMapper = ConstantColumnMapper(column_name="triglycerides",hpo_id="HP:0002155", hpo_label="Hypertriglyceridemia")
column_mapper_list.append(htMapper)
# htMapper.preview_column(df)

In [12]:
# Hypercholesterolemia HP:0003124
cholMapper = ConstantColumnMapper(column_name='cholesterol.total',hpo_id="HP:0003124", hpo_label="Hypercholesterolemia")
column_mapper_list.append(cholMapper)
# cholMapper.preview_column(df)

In [13]:
# Decreased HDL cholesterol concentration HP:0003233
hdlMapper = ConstantColumnMapper(column_name="HDL",hpo_id="HP:0003233", hpo_label="Decreased HDL cholesterol concentration")
column_mapper_list.append(hdlMapper)
# hdlMapper.preview_column(df)

In [14]:
# Hyperglycemia HP:0003074 -- assume upper limit of normal of 5.6 mmol/L
glucoseMapper = ConstantColumnMapper(column_name='blood.glucose',hpo_id="HP:0003074", hpo_label="Hyperglycemia")
column_mapper_list.append(glucoseMapper)
#glucoseMapper.preview_column(df)

In [15]:
# Elevated hemoglobin A1c HP:0040217 -- assume normal range at 4.5-5.7%
hba1cg_d = {'7.2': 'Elevated hemoglobin A1c',
 '7.0': 'Elevated hemoglobin A1c',
 '9.5': 'Elevated hemoglobin A1c'}
hba1cgMapper = OptionColumnMapper(column_name='HBA1Cg',concept_recognizer=hpo_cr, option_d=hba1cg_d)
column_mapper_list.append(hba1cgMapper)
hba1cgMapper.preview_column(df)

,mapping,count
0,Elevated hemoglobin A1c (HP:0040217) (observed),3


In [16]:
ageMapper = AgeColumnMapper.by_year(column_name="Age")
#ageMapper.preview_column(df)
sexMapper = SexColumnMapper(male_symbol="M", female_symbol="F", column_name="Sex")
#sexMapper.preview_column(df)

In [17]:
onsetMapper = AgeColumnMapper.hpo_onset(column_name="Onset term")
onsetMapper.preview_column(df)

,original column contents,age
0,Juvenile onset,Juvenile onset
1,Adult onset,Adult onset


In [18]:
LMNA_transcript = "NM_170707.4"
vman = VariantManager(df=df, individual_column_name="Patient",allele_1_column_name="NM_170707.4",
                      gene_symbol="LMNA", transcript=LMNA_transcript)

In [19]:
var_d = vman.get_variant_d()

In [21]:
varMapper = VariantColumnMapper(variant_d=var_d,
                                variant_column_name="NM_170707.4",
                                default_genotype="heterozygous")
encoder = CohortEncoder(df=df, 
                        hpo_cr=hpo_cr, 
                        column_mapper_list=column_mapper_list, 
                        individual_column_name="Patient", 
                        age_at_last_encounter_mapper=ageMapper, 
                        age_of_onset_mapper=onsetMapper,
                        sexmapper=sexMapper,
                        variant_mapper=varMapper,
                        metadata=metadata)
omim_id = "OMIM:151660"
omim_label = "Lipodystrophy, familial partial, type 2"
lipodystrophy = Disease(disease_id=omim_id, disease_label=omim_label)
encoder.set_disease(disease=lipodystrophy)

## Excluded phenotypes
The authors write that No findings of other laminopathies were present in any patient.
Therefore, we exclude several characteristic findings

- Proptosis (HP:0000520); 
- Failure to thrive (HP:0001508); 
- Micrognathia (HP:0000347); 
- Alopecia (HP:0001596); 
- Short stature (HP:0004322); 
- Scleroderma (HP:0100324); 
- Prominent superficial blood vessels (HP:0007394); 
- Mottled pigmentation (HP:0001070)
- Progeroid facial appearance HP:0005328
- Cardiomyopathy (HP:0001638)
- Atrioventricular block (HP:0001678)

In [22]:
individuals = encoder.get_individuals()
for i in individuals:
    proptosis = HpTerm(hpo_id="HP:0000520", label="Proptosis", observed=False)
    i.add_hpo_term(proptosis)
    micrognathia = HpTerm(hpo_id="HP:0000347", label="Micrognathia", observed=False)
    i.add_hpo_term(micrognathia)
    sqfat = HpTerm(hpo_id="HP:0003758", label="Reduced subcutaneous adipose tissue", observed=False)
    i.add_hpo_term(sqfat)
    alopecia = HpTerm(hpo_id="HP:0001596", label="Alopecia", observed=False)
    i.add_hpo_term(alopecia)
    vessels = HpTerm(hpo_id="HP:0007394", label="Prominent superficial blood vessels", observed=False)
    i.add_hpo_term(vessels)
    ftt = HpTerm(hpo_id="HP:0001508", label="Failure to thrive", observed=False)
    i.add_hpo_term(ftt)
    sst = HpTerm(hpo_id="HP:0004322", label="Short stature", observed=False)
    i.add_hpo_term(sst)
    bmt = HpTerm(hpo_id="HP:0004349", label="Reduced bone mineral density", observed=False)
    i.add_hpo_term(bmt)
    prog = HpTerm(hpo_id="HP:0005328", label="Progeroid facial appearance", observed=False)
    i.add_hpo_term(bmt)
    cmp = HpTerm(hpo_id="HP:0001638", label="Cardiomyopathy", observed=False)
    i.add_hpo_term(cmp)
    avb = HpTerm(hpo_id="HP:0001678", label="Atrioventricular block", observed=False)
    i.add_hpo_term(avb)

In [23]:
cvalidator = CohortValidator(cohort=individuals, ontology=hpo_ontology, min_hpo=1, allelic_requirement=AllelicRequirement.MONO_ALLELIC)
qc = QcVisualizer(cohort_validator=cvalidator)
display(HTML(qc.to_summary_html()))

Level,Error category,Count


In [24]:
individuals = cvalidator.get_error_free_individual_list()
table = PhenopacketTable(individual_list=individuals, metadata=metadata)
display(HTML(table.to_html()))

Individual,Disease,Genotype,Phenotypic features
A (FEMALE; P46Y),"Lipodystrophy, familial partial, type 2 (OMIM:151660)",NM_170707.4:c.688G>A (heterozygous),Obesity (HP:0001513); Increased facial adipose tissue (HP:0000287); Loss of subcutaneous adipose tissue in limbs (HP:0003635); Lipodystrophy (HP:0009125); Type II diabetes mellitus (HP:0005978); Hypertriglyceridemia (HP:0002155); Hypercholesterolemia (HP:0003124); Decreased HDL cholesterol concentration (HP:0003233); Hyperglycemia (HP:0003074); Elevated hemoglobin A1c (HP:0040217); excluded: Reduced bone mineral density (HP:0004349); excluded: Cardiomyopathy (HP:0001638); excluded: Reduced subcutaneous adipose tissue (HP:0003758); excluded: Acanthosis nigricans (HP:0000956); excluded: Alopecia (HP:0001596); excluded: Atrioventricular block (HP:0001678); excluded: Micrognathia (HP:0000347); excluded: Proptosis (HP:0000520); excluded: Prominent superficial blood vessels (HP:0007394); excluded: Short stature (HP:0004322); excluded: Failure to thrive (HP:0001508)
B (FEMALE; P50Y),"Lipodystrophy, familial partial, type 2 (OMIM:151660)",NM_170707.4:c.1195C>T (heterozygous),Obesity (HP:0001513); Increased facial adipose tissue (HP:0000287); Loss of subcutaneous adipose tissue in limbs (HP:0003635); Lipodystrophy (HP:0009125); Type II diabetes mellitus (HP:0005978); Hypertriglyceridemia (HP:0002155); Hypercholesterolemia (HP:0003124); Decreased HDL cholesterol concentration (HP:0003233); Hyperglycemia (HP:0003074); Elevated hemoglobin A1c (HP:0040217); excluded: Reduced bone mineral density (HP:0004349); excluded: Cardiomyopathy (HP:0001638); excluded: Reduced subcutaneous adipose tissue (HP:0003758); excluded: Acanthosis nigricans (HP:0000956); excluded: Alopecia (HP:0001596); excluded: Atrioventricular block (HP:0001678); excluded: Micrognathia (HP:0000347); excluded: Proptosis (HP:0000520); excluded: Prominent superficial blood vessels (HP:0007394); excluded: Short stature (HP:0004322); excluded: Failure to thrive (HP:0001508)
C (MALE; P75Y),"Lipodystrophy, familial partial, type 2 (OMIM:151660)",NM_170707.4:c.1718C>T (heterozygous),Obesity (HP:0001513); Increased facial adipose tissue (HP:0000287); Loss of subcutaneous adipose tissue in limbs (HP:0003635); Lipodystrophy (HP:0009125); Type II diabetes mellitus (HP:0005978); Hypertriglyceridemia (HP:0002155); Hypercholesterolemia (HP:0003124); Decreased HDL cholesterol concentration (HP:0003233); Hyperglycemia (HP:0003074); Elevated hemoglobin A1c (HP:0040217); excluded: Reduced bone mineral density (HP:0004349); excluded: Cardiomyopathy (HP:0001638); excluded: Reduced subcutaneous adipose tissue (HP:0003758); excluded: Acanthosis nigricans (HP:0000956); excluded: Alopecia (HP:0001596); excluded: Atrioventricular block (HP:0001678); excluded: Micrognathia (HP:0000347); excluded: Proptosis (HP:0000520); excluded: Prominent superficial blood vessels (HP:0007394); excluded: Short stature (HP:0004322); excluded: Failure to thrive (HP:0001508)


In [25]:
Individual.output_individuals_as_phenopackets(individual_list=individuals, metadata=metadata)

We output 3 GA4GH phenopackets to the directory phenopackets
